# U-Net based Image Segmentation for locating Radion Frequencey Interference Patches on Sentinel-1 images 

## Initialisations

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# # Initliasations

# Keras deep learning library
import keras
from keras.models import load_model, Model
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, concatenate, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import losses

In [ ]:
# Geospatial libraries to handle georeferenced raster and vector data
import fiona
from fiona import collection
import rasterio
import geopandas as gpd
from shapely.geometry import LineString, mapping, shape
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.transform import Affine
from rasterio.warp import reproject, Resampling
from rasterio import windows

In [ ]:
# Matplotlib for data plotting
from matplotlib import pyplot as plt

# Scikit learn and scikit image libraries for machine learning and computer vision tools.
from skimage import io, exposure, measure
from sklearn.metrics import jaccard_similarity_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import StandardScaler

In [ ]:
# Useful python libaries for manipulating data
import os
import pickle
import itertools
import glob
import re
import math

# Pandas to bring dataframe structure
import pandas as pd

# Numpy for numerical processing and arrays
import numpy as np

# Initializse a random number generator to ensure results are reproducible
np.random.seed(7)
# turn warnig off
np.warnings.filterwarnings('ignore')
# print floating points as using fixed point notation i.e. not scientific
np.set_printoptions(suppress=True)
% matplotlib inline

In [ ]:
# set some useful paths
# RAW INPUTS
#root = 'i:/DSA/Detect_RFI/'
root = '/Volumes/NewVolume/DSA/Detect_RFI/'
s1_train_dir = os.path.join(root, 'DATA/S1_TRAIN/SPLIT')  # full res input S1 ARD for training
s1_test_dir = os.path.join(root, 'DATA/S1_TEST/SPLIT')  # full res input S1 ARD for testing

# perhaps would be an idea to transform ARD into some generic metric surface like normalised (scale-min max)slope, curvature
# so that model works for images from anywhere
labels_dir = os.path.join(root, 'DATA/LABELS')  # mask polygons

# DERIVED INPUTS
image_chips_dir = os.path.join(root, 'DATA/IMAGE_CHIPS')  # pre-procssed image tiles and mask root folder
model_dir = os.path.join(root, 'MODELS')  # contains the machine learning models
predictions_dir = os.path.join(root, 'PREDICTIONS')

## Load up the masks and store them in a dictionary indexed by the Sentinel-1 tile name

In [ ]:
# Create training data polygons from masks shapefile
# open the masks polygons and create an array of geometries for unique tilename
def makeMasks(shapeFileName,labelFieldName):
    LU_LABELS = {}
    with fiona.open(shapeFileName, 'r') as landcovers:
        for feature in landcovers:
            LU_LABEL = feature['properties'][labelFieldName]
            mask = feature["geometry"]
            if LU_LABEL in LU_LABELS:
                oldArray = LU_LABELS[LU_LABEL]
                oldArray.append(mask)
            else:
                LU_LABELS[LU_LABEL] = [mask]
    return LU_LABELS

In [ ]:
# read land cover polygon  into geometry arrays that rasterio will use to create masks
labels_shp = os.path.join(labels_dir, 'RFI_Masks_Final.shp')
LU_LABELS = makeMasks(labels_shp,'TILENAME')

## Pre-Processing Input Data i.e. Prepare Image Chips and Mask the non-data/data areas

In [ ]:
def maskImage(input_image_name,allPolygons):
    try:
        temp_file_name = input_image_name.replace(".tif","_temp.tif")
        maskDone = False
        with rasterio.open(input_image_name) as src:
            mask_image, mask_transform = rasterio.mask.mask(src,allPolygons,nodata=0)
            mask_meta = src.meta.copy()                   
            # skip this tile if it only has nodata otherwise
            # we are overfeeding nodata
            uniqueLabels = np.unique(mask_image[0,:,:])
            if len(uniqueLabels) < 2 and uniqueLabels[0] ==0:                
                maskDone= input_image_name
            else:
                with rasterio.open(temp_file_name, 'w', **mask_meta) as dest:
                    dest.write(mask_image)
                    maskDone = temp_file_name
        
        if (maskDone == input_image_name):
            os.remove(input_image_name)
        elif (maskDone == temp_file_name):
            os.remove(input_image_name)
            os.rename(temp_file_name,input_image_name)
        return True    
    except Exception as e:
        print(e)
        return maskDone 

In [ ]:
# function by cate to create 3D tiles 
def get_tile_images(arr, newshape):
    oldshape = np.array(arr.shape)
    repeats = (oldshape / newshape).astype(int)
    tmpshape = np.column_stack([repeats, newshape]).ravel()
    order = np.arange(len(tmpshape))
    order = np.concatenate([order[::2], order[1::2]])
    #pdb.set_trace()
    # newshape must divide oldshape evenly or else ValueError will be raised
    return arr.reshape(tmpshape).transpose(order).reshape(-1, *newshape)

In [ ]:
# read the s1 ard image, apply the standard deviation scaling
# it to the ARD
def scaleImage(S1_ImageName, tileYSize, tileXSize):
    with rasterio.open(S1_ImageName) as S1_src:
        newHeight = int(S1_src.height / tileYSize) * tileYSize
        newWidth = int(S1_src.width / tileXSize) * tileXSize
        if (newHeight < tileYSize or newWidth < tileXSize):
            print("newhieght or newwidth are smaller than tile sizes")
            return None,None,None,None
        
        if (newHeight < newWidth):
            newWidth = newHeight
        elif (newWidth < newHeight):
            newHeight = newWidth
    
        S1_image = S1_src.read(window=((0, newWidth), (0, newHeight)))
        
#        if (np.isnan(S1_image.max()) or np.isnan(S1_image.min())):
#            print("nan found")
#            return None,None,None,None
#        if (S1_image.max()<0.0000001 and S1_image.min()<0.0000001):
#            print("near zero found")
#            return None,None,None,None
     
#        print(str(S1_image.max()))
#        print(str(S1_image.min()))
#        print (S1_image.shape)
#        S1_image = exposure.equalize_hist(S1_image)        
        # StandardScaler(copy=True, with_mean=False, with_std=True).fit_transform(S1_image)        
        # show(S1_image)
        
        crs = S1_src.crs
        out_metadata = S1_src.meta.copy()
        out_transform = S1_src.transform   
        
    return S1_image, out_metadata, out_transform, crs

In [ ]:
def prepImages(s1_images_dir,image_chips_dir,allPolygons):
    # read the s1 ARD image names that we have and store them in a dictionary
    
    for filename in os.listdir(s1_images_dir):
        if filename.lower().endswith(".tif"):        
            # iterate through the S1 ARD images
            # create image chips
            
            tileName = filename[:4] 
            if tileName in allPolygons:            
                RFI_Masks = allPolygons[tileName]
            else:
                continue
            
            s1_image = os.path.join(s1_images_dir,filename)   
            print("Processing: "+s1_image)
            
            scaled_image, inputMetadata, inputTransform, crs = scaleImage(s1_image,256,256) #1
            if (scaled_image is None):
                continue
            
            ULX = inputTransform.c
            ULY = inputTransform.f
            cellSizeX = inputTransform.a
            cellSizeY = inputTransform.e  
            
            scaled_image_tiled = get_tile_images(scaled_image,(1,256,256)) #2
            
            # write image chips
            numtiles,numbands,rows,cols = scaled_image_tiled.shape
            rowOffset = 0
            blockSize = int(math.sqrt(numtiles))
            for tileIndexAlongY in range(0,blockSize):
                tileULY = ULY + (tileIndexAlongY * cellSizeY * 256)
                for tileIndexAlongX in range(0,blockSize):
                    tileULX = ULX + (tileIndexAlongX * cellSizeX * 256)
                    # print(str(tileULX)+","+str(tileULY))
                    globalTileIndex = tileIndexAlongY + tileIndexAlongX + rowOffset
                    aTileImage = scaled_image_tiled[globalTileIndex,:,:]
                           
                    # aTileImage = np.expand_dims(aTileImage,0)
                    tileMetadata = inputMetadata.copy()
                    tileTransform = Affine(cellSizeX,inputTransform.b,tileULX,inputTransform.d,cellSizeY,tileULY)                

                    # two more bands have been added                
                    tileMetadata.update({'driver': 'GTiff',
                                         'dtype': 'float32',
                                         'count': 1,
                                         'height': 256,
                                         'width': 256,
                                         'crs': crs,
                                         'transform': tileTransform,
                                         'nodata':0})              
                    tileFileName = os.path.basename(s1_image).replace(".tif","_"+str(globalTileIndex).zfill(5)+".tif")
                    tileFullName = os.path.join(image_chips_dir, tileFileName)
                    with rasterio.open(tileFullName, 'w', **tileMetadata) as dest:
                        dest.write(aTileImage)
                
                    #now mask nodata areas in the image
                    maskDone = maskImage(tileFullName,RFI_Masks)
                    if (not maskDone):
                        print("failed")
                    
                rowOffset = rowOffset + (blockSize-1)

In [ ]:
def maskImages(images_dir,labels_chips_dir,LU_LABELS):
    for filename in os.listdir(images_dir):
        if filename.endswith(".tif"):
            image_path = os.path.join(images_dir,filename)
            #find the image tile name
            tileName = filename[:4]
            #create a list of output ndarrays
            mask_array = []
            #iterate through the mask vectors
            with rasterio.open(image_path) as src:           
                masks = LU_LABELS[tileName]
                mask_image, mask_transform = rasterio.mask.mask(src,masks,nodata=0)
                mask_meta = src.meta.copy()
                mask_image[mask_image != 0] = 1

                mask_image = mask_image[0].astype('uint8')
                mask_image = np.expand_dims(mask_image, 0)
            
                #save the final mask image
                mask_meta.update({'driver': 'GTiff',
                             'dtype': 'uint8',
                             'count': 1,
                             'height': mask_image.shape[1],
                             'width': mask_image.shape[2],
                             'crs': src.crs,
                             'transform': mask_transform})
                label_file_name = os.path.join(labels_chips_dir,filename.replace(".tif","_labels.tif"))
                with rasterio.open(label_file_name, 'w', **mask_meta) as dest:
                    dest.write(mask_image)

## Convolutional neural network
## The last approach will implement Unet - a popular convolutional neural network for image classifcation

In [ ]:
###Set up the training data

In [ ]:
# preprocess the training RGB and CIR images to resample CIR, extract NIR band, append NIR band to RGB, split into small tiles
train_chips_dir = os.path.join(image_chips_dir, 'TRAIN')
prepImages(s1_train_dir,train_chips_dir, LU_LABELS)

In [ ]:
# using rasterio mask function to mask pixels using LU_LABEL in the training images
train_images_dir = os.path.join(image_chips_dir,'TRAIN')
train_labels_chips_dir = os.path.join(train_images_dir,'LABELS')

In [ ]:
maskImages(train_images_dir,train_labels_chips_dir,LU_LABELS)

In [ ]:
# set paths to the training data
image_chips = train_images_dir
label_chips = train_labels_chips_dir

In [ ]:
# Class to store data and respective labels
# imagese are split into halves..first half used for training, and remaining half into validation
class train_data():
    
    def __init__(self, image, label):
        self.image = []
        self.label = []
        for file in os.listdir(image):
            if file.endswith(".tif"):
                label_file= os.path.join(label,file.replace(".tif","_labels.tif"))
                imgArray = io.imread(image+"/"+file,0)
                #print(imgArray.shape)
                imgArray = np.expand_dims(imgArray,2)
                #print(imgArray.shape)
                self.image.append(imgArray)
                lblArray = io.imread(label_file,0)
                lblArray = np.expand_dims(lblArray,2)
                self.label.append(lblArray)
    
    # training half
    def get_image(self):
        return np.array(self.image[:int(len(self.image)/2)])

    def get_label(self):
        return np.array(self.label[:int(len(self.image)/2)])
    
    # validation half
    def get_validation_image(self):
        return np.array(self.image[int(len(self.image)/2):])
    
    def get_validation_label(self):
        return np.array(self.label[int(len(self.image)/2):])
        
    def set_image(self, new_images):
        self.image = new_image
    
    def set_label(self,new_label):
        self.label = new_label

In [ ]:
# run the training data creation
train_set = train_data(image_chips, label_chips)

In [ ]:
# access the training set
train_images = train_set.get_image()
train_label = train_set.get_label()
# access the validation set
validation_image = train_set.get_validation_image()
validation_label = train_set.get_validation_label()
# one hot encode the labels
train_label_encoded = to_categorical(train_label, num_classes=2)
validation_label_encoded = to_categorical(validation_label, num_classes=2)

In [ ]:
show(train_images[44,:,:,:])
show(train_label[44,:,:,:])

In [ ]:
# check that Keras expects the bands to be passed last - i.e. the data is shaped (256, 256, 12)
keras.backend.image_data_format()

In [ ]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [ ]:
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def unet_zhixuhao(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(2, (1, 1), activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [ ]:
# define the Unet architecture
def unet_cate(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    # use softmax in order to only output one class per pixel (unlike sigmoid which can have multiclass)
    conv10 = Conv2D(2, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
        
    #model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=Adam(lr=1e-9), loss='binary_crossentropy', metrics=['accuracy'])
    # see if a different loss and metrics can improve e.g. sorenson-dice
    # model.compile(optimizer=Adam(lr=1e-9), loss=dice_coef_loss, metrics=[dice_coef])
    
    model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    
    return model

In [ ]:
# train the model
model = unet_cate()
model.fit(train_images, train_label_encoded, validation_data=(validation_image, validation_label_encoded), epochs=100,
          batch_size=8, shuffle=True)
# save the model and weights
model.save(os.path.join(model_dir, '20181014_Unet_100epoch_RFI_dice.h5'))

In [ ]:
#model = load_model(os.path.join(model_dir, '20180905_Unet_10epoch.h5'))

# lower the learning rate
#lr = 0.0001
#K.set_value(model.optimizer.lr, lr)

#model.fit(train_images, train_label_encoded, validation_data=(validation_image, validation_label_encoded), epochs=10,
#          batch_size=10, shuffle=True)
#model.save(os.path.join(model_dir, '20180905_Unet_20epoch.h5'))

In [ ]:
#model = load_model(os.path.join(model_dir, '20180905_Unet_20epoch.h5'))

# try increasing the learning rate as little improvement was seen in previous epochs
#lr = 0.001
#K.set_value(model.optimizer.lr, lr)

#model.fit(train_images, train_label_encoded, validation_data=(validation_image, validation_label_encoded), epochs=10,
#          batch_size=8, shuffle=True)
#model.save(os.path.join(model_dir, '20180905_Unet_30epoch.h5'))

## Test the accuracy of the predictions using known test cases

In [ ]:
# preprocess the test RGB and CIR images to resample CIR, extract NIR band, append NIR band to RGB, split into small tiles
test_chips_dir = os.path.join(image_chips_dir, 'TEST')
prepImages(s1_test_dir,test_chips_dir,LU_LABELS)

In [ ]:
test_images_dir = os.path.join(image_chips_dir,'TEST')
test_labels_chips_dir = os.path.join(test_images_dir,'LABELS')

In [ ]:
# using rasterio mask function to mask pixels using LU_LABEL in the test images
maskImages(test_images_dir,test_labels_chips_dir,LU_LABELS)

In [ ]:
# read in the trained model
unet_model = load_model(os.path.join(model_dir, '20181014_Unet_100epoch_RFI_dice.h5'))

In [ ]:
# load test data
test_image_chips = test_images_dir 
test_label_chips = test_labels_chips_dir 

class test_data():
    
    def __init__(self, image, label):
        self.image = []
        self.label = []
        self.filename = [] # store the filename because we can use it later to recreate the predicted image for it
        
        for file in os.listdir(image):
            
            if file.endswith(".tif"):
                label_file= os.path.join(label,file.replace(".tif","_labels.tif"))
                imgArray = io.imread(image+"/"+file,0)
                imgArray = np.expand_dims(imgArray, 2) # this is done to meet keras channel order needs
                self.image.append(imgArray)
                lblArray = io.imread(label_file,0)
                lblArray = np.expand_dims(lblArray,2)
                self.label.append(lblArray)
                self.filename.append(file)
               
    def get_image(self):
        return np.array(self.image[:int(len(self.image))])

    def get_label(self):
        return np.array(self.label[:int(len(self.image))])
        
    def set_image(self, new_images):
        self.image = new_image
    
    def set_label(self,new_label):
        self.label = new_label

In [ ]:
# create the test set
test_set = test_data(test_image_chips, test_label_chips)
# access the test images and labels
test_images = test_set.get_image()
test_label = test_set.get_label()
# one-hot-encode the labels
test_label_encoded = to_categorical(test_label, num_classes=2)

In [ ]:
# make predictions on the test set using the trained model
test_predict = unet_model.predict(test_images, verbose=1)

In [ ]:
# decode the predictions
test_predict_decoded = np.argmax(test_predict, axis=3)

In [ ]:
# to show the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
 
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
 
# to calculate all metrics
def accuracy_statistics(true_class, predicted_class):
   
    # classification report
    #target_names = ["Class {}".format(i) for i in range(3)]
    print('Classification Report')
    print(classification_report(true_class, predicted_class))
   
    # jaccard
    print('Jaccard Score:')
    print(jaccard_similarity_score(true_class, predicted_class))
   
    # confusion matrix
    # plt.figure()
    print ('Confusion Matrix')
    #print(confusion_matrix(true_class, predicted_class))
    cnf_matrix = confusion_matrix(true_class, predicted_class)

    plot_confusion_matrix(cnf_matrix, classes=['NoData', 'RFI Area'],
                          title='Confusion Matrix')
    plot_confusion_matrix(cnf_matrix, classes=['NoData', 'RFI Area'], normalize=True,
                          title='Normalized Confusion Matrix')

In [ ]:
# generate accuracy statistics
accuracy_statistics(test_label.ravel(), test_predict_decoded.ravel())

In [ ]:
# iterate through the test_images, find the related predicted arrays, find the geotransformation of the test_image it came from
# and the save the predicted array as an image using the geotransform
for tileNum in range(len(test_predict_decoded)):
    imgFileName = test_set.filename[tileNum]
    imageFullName = os.path.join(test_image_chips,imgFileName)
    with rasterio.open(imageFullName) as src:
        out_transform = src.transform
        out_meta = {'driver': 'GTiff',
                    'dtype': 'uint8',
                    'count': 1,
                    'height': 256,
                    'width': 256,
                    'crs': src.crs,
                    'transform': out_transform,
                    'nodata':0}
        predictionFileName = os.path.join(predictions_dir, 'UNET',imgFileName.replace(".tif","_prediction.tif"))
        with rasterio.open(predictionFileName, 'w', **out_meta) as dest:
                    im_prediction = test_predict_decoded[tileNum,:,:]
                    im_prediction = np.expand_dims(im_prediction, 0)
                    im_prediction = im_prediction.astype('uint8')
                    dest.write(im_prediction) 